In [1]:
from typing import Tuple, Optional, Dict

from PIL import Image
import pandas as pd
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop

from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from typing import Dict, List, Sequence
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import numpy as np
from skimage import color


def mask2rle(img) -> str:
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle, shape=(3000, 3000)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.int32)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


def resize_image(input_image: np.ndarray, target_shape: Tuple[int, int]) -> np.ndarray:
    image_original = Image.fromarray(input_image)
    reshaped_image = image_original.resize(target_shape)
    array_transformed = np.array(reshaped_image)
    return array_transformed


class TrainDataset(Dataset):
    def __init__(self, df_train: pd.DataFrame, image_shape: Tuple[int, int] = (256, 256),
                 dataset_dir: str = "/content/"):
        self.df_train = df_train
        self.dataset_dir = dataset_dir
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.image_shape = image_shape

    def __getitem__(self, index: int):
        h, w = self.df_train['img_height'].iloc[index], self.df_train['img_width'].iloc[index]
        path = f"{self.dataset_dir}train_images/{self.df_train['id'].iloc[index]}.tiff"
        image = Image.open(path)
        array = resize_image(np.array(image), target_shape=self.image_shape)
        image_tensor = torch.FloatTensor(array).to(self.device).permute(2, 0, 1)
        output_mask = rle2mask(mask_rle=self.df_train['rle'].iloc[index], shape=(h, w))
        output_mask_resized = resize_image(input_image=output_mask, target_shape=self.image_shape)
        target_tensor = torch.FloatTensor(output_mask_resized).to(self.device).unsqueeze(0)
        return {"input": image_tensor,
                "target": target_tensor}

    def __len__(self):
        return self.df_train.shape[0]


class TestDataset(Dataset):
    def __init__(self, df_test: pd.DataFrame, image_shape: Tuple[int, int] = (256, 256),
                 dataset_dir: str = "/content/"):
        self.df_test = df_test
        self.dataset_dir = dataset_dir
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.image_shape = image_shape

    def __getitem__(self, index: int):
        h, w = self.df_test['img_height'].iloc[index], self.df_test['img_width'].iloc[index]
        path = f"{self.dataset_dir}test_images/{self.df_test['id'].iloc[index]}.tiff"
        image = Image.open(path)
        array = resize_image(np.array(image), target_shape=self.image_shape)
        image_tensor = torch.FloatTensor(array).to(self.device).permute(2, 0, 1)
        return {"input": image_tensor}

    def __len__(self):
        return self.df_test.shape[0]


class PreTrainDataset(Dataset):
    def __init__(self, df_train: pd.DataFrame, image_shape: Tuple[int, int] = (256, 256),
                 dataset_dir: str = "/content/", transforms: Optional = None):
        self.df_train = df_train
        self.dataset_dir = dataset_dir
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.image_shape = image_shape
        if transforms is None:
            self.transform = nn.Sequential(
                RandomHorizontalFlip(),
                RandomResizedCrop((self.image_shape[0] // 2, self.image_shape[1] // 2))
            ).to(self.device)
        else:
            self.transform = transforms

    def _transform_tensors(self, result: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        input_image = result['input']
        input_target = result['target']
        concat_result = torch.cat((input_image, input_target), dim=0)
        transformed_result = self.transform(concat_result)
        return {"input": transformed_result[:3],
                "target": transformed_result[3].unsqueeze(0)}

    def __getitem__(self, index: int):
        h, w = self.df_train['img_height'].iloc[index], self.df_train['img_width'].iloc[index]
        path = f"{self.dataset_dir}train_images/{self.df_train['id'].iloc[index]}.tiff"
        image = Image.open(path)
        array = resize_image(np.array(image), target_shape=self.image_shape)
        image_tensor = torch.FloatTensor(array).to(self.device).permute(2, 0, 1)
        output_mask = rle2mask(mask_rle=self.df_train['rle'].iloc[index], shape=(h, w))
        output_mask_resized = resize_image(input_image=output_mask, target_shape=self.image_shape)
        target_tensor = torch.FloatTensor(output_mask_resized).to(self.device).unsqueeze(0)
        return self._transform_tensors({"input": image_tensor, "target": target_tensor})
    def __len__(self):
        return self.df_train.shape[0]


class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.neck = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.upconv2 = nn.Sequential(
            nn.Conv2d(128, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Dropout(0.1)
        )
        self.upconv1 = nn.Sequential(
            nn.Conv2d(96, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 1, 3, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, input_image):
        hidden1 = self.conv1(input_image)  # 64, 128, 128
        hidden2 = self.conv2(hidden1)  # 64, 64, 64
        output = self.neck(hidden2)
        output = torch.cat((output, hidden2), dim=1)
        output = self.upconv2(output)
        output = torch.cat((output, hidden1), dim=1)
        return self.upconv1(output)


def train(model: nn.Module,
          optimizer: torch.optim,
          train_loader: DataLoader,
          criterion: nn.Module,
          device: torch.device,
          num_epochs: int = 10,
          log_window: int = 20):
    for epoch in trange(num_epochs):
        pbar = tqdm(train_loader)
        sum_loss, cnt_loss = 0, 0
        cur_sum_loss, cur_cnt_loss = 0, 0
        for batch in pbar:
            optimizer.zero_grad()
            input_tensor = batch['input'].to(device)
            target_tensor = batch['target'].to(device)
            output = model(input_tensor)
            loss = criterion(output, target_tensor)
            loss.backward()
            sum_loss += loss.item()
            cnt_loss += 1
            cur_sum_loss += loss.item()
            cur_cnt_loss += 1
            description = f"Mean total loss: {round(sum_loss / cnt_loss, 3)} | mean window loss: {round(cur_sum_loss / cur_cnt_loss, 3)}"
            pbar.set_description(description)
            if cur_cnt_loss == log_window:
                cur_sum_loss, cur_cnt_loss = 0, 0
            optimizer.step()
        torch.save(model.state_dict(), f"output_model_{epoch}")


def load_weights(model: nn.Module, PATH: str) -> nn.Module:
    model.load_state_dict(torch.load(PATH))
    return model


@torch.no_grad()
def inference_valid(model: nn.Module,
                    valid_loader: DataLoader,
                    device: torch.device,
                    criterion: nn.Module = nn.BCELoss(),
                    threshold: float = 0.5,
                    eps: float = 1e-5
                    ) -> Dict[str, float]:
    sum_loss, cnt_loss = 0, 0
    sum_acc, cnt_acc = 0, 0
    for batch in valid_loader:
        input_tensor = batch['input'].to(device)
        output = model(input_tensor)

        output_np = output.detach().cpu().numpy().reshape(-1) > threshold
        target_np = batch['target'].detach().cpu().numpy().reshape(-1) 
        
        sum_acc += sum(output_np == target_np)
        cnt_acc += len(target_np)

        loss = criterion(output, batch['target'].to(device))
        sum_loss += loss.item()
        cnt_loss += 1

    return {"loss": sum_loss / cnt_loss,
            "accuracy": sum_acc / cnt_acc}


@torch.no_grad()
def get_rle_predictions(model: nn.Module,
                        test_dataset: Dataset,
                        device: torch.device,
                        heights: Sequence[int],
                        widths: Sequence[int],
                        threshold: float = 0.5
                        ) -> List[str]:
    answer = []
    for i, batch in enumerate(test_dataset):
        input_tensor = batch['input'].unsqueeze(0).to(device)
        output = model(input_tensor).squeeze(0).squeeze(0).cpu().detach().numpy() > threshold
        output = resize_image(output, (heights[i], widths[i]))
        rle = mask2rle(output)
        answer.append(rle)
    return answer



def show_masked_img(img, mask, title=''):
    mask = (mask - mask.min()) / (mask.max() - mask.min())
    mask = np.nan_to_num(mask)
    mask = np.round(mask)
    mask = mask.reshape(img.shape[:2])

    fig, ax = plt.subplots(1, 3, figsize=(9, 3))
    fig.suptitle(title, fontsize=16)

    ax[0].imshow(mask)
    ax[0].set_title('Mask')
    ax[1].imshow(img)
    ax[1].set_title('Image')
    ax[2].imshow(color.label2rgb(mask, img,
                                 bg_label=0, bg_color=(1., 1., 1.), alpha=0.25))
    ax[2].set_title('Masked Image')
    plt.show()


def plot_batch(output: torch.Tensor, target: torch.Tensor):
    output_cpu = output.permute(0, 2, 3, 1).cpu().detach().numpy()
    target_cpu = target.permute(0, 2, 3, 1).cpu().detach().numpy()
    for img, msk in zip(target_cpu, output_cpu):
        show_masked_img(img, msk, "Sample")


import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class TransUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.01),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1)

        )

        self.encoder = nn.Transformer(nhead=4).encoder

        self.up1 = nn.Sequential(
            nn.Conv2d(512, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.01),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(256, 128, 3, 1, 1),
            nn.Dropout(0.1)
        )

        self.up2 = nn.Sequential(
            nn.Conv2d(256, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.01),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(128, 64, 3, 1, 1),
            nn.Dropout(0.1)
        )
        self.last = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(128, 1, 3, 1, 1)
        )
        self.sigmoid = nn.Sigmoid()
        self.pe = PositionalEncoding(512)

    def forward(self, input: torch.Tensor):
        hidden1 = self.conv1(input) # b, 64, 128, 128
        hidden2 = self.conv2(hidden1) # b, 128, 64, 64
        hidden3 = self.conv3(hidden2) # b, 256, 32, 32
        batch_size = hidden3.shape[0]
        image_shape = tuple(hidden3.shape)
        transformer_patch = hidden3.reshape(batch_size, -1, 512)
        transformer_patch = self.pe(transformer_patch)
        output_transformer = self.encoder(transformer_patch)
        unpatched_batch = output_transformer.reshape(image_shape) # b, 256, 32, 32
        output = torch.cat((unpatched_batch, hidden3), 1) # b, 512, 32, 32
        output = self.up1(output) # b, 128, 64, 64
        output = torch.cat((output, hidden2), 1) # b, 256, 64, 64
        output = self.up2(output) # b, 64, 128, 128
        output = torch.cat((output, hidden1), 1) # b, 64, 128, 128
        output = self.last(output)
        return self.sigmoid(output)


class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.neck = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.upconv2 = nn.Sequential(
            nn.Conv2d(128, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Dropout(0.1)
        )
        self.upconv1 = nn.Sequential(
            nn.Conv2d(96, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 1, 3, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, input_image):
        hidden1 = self.conv1(input_image)  # 64, 128, 128
        hidden2 = self.conv2(hidden1)  # 64, 64, 64
        output = self.neck(hidden2)
        output = torch.cat((output, hidden2), dim=1)
        output = self.upconv2(output)
        output = torch.cat((output, hidden1), dim=1)
        return self.upconv1(output)


from torch.utils.data import DataLoader, Subset, Dataset
from torch.utils.data import random_split

def split_dataset(dataset: Subset, train_part: float) -> Tuple[Subset, Subset]:
    train_len = int(len(dataset) * train_part)
    lengths = [train_len, len(dataset) - train_len]
    train, test = random_split(dataset, lengths=lengths)
    return (train, test)


In [2]:
import pandas as pd
df_train = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
df_train

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female
...,...,...,...,...,...,...,...,...,...,...
346,9517,kidney,HPA,3000,3000,0.4,4,1611763 11 1614753 29 1617750 35 1620746 43 16...,61.0,Male
347,9769,kidney,HPA,3070,3070,0.4,4,4030400 28 4033466 34 4036526 48 4039594 54 40...,28.0,Male
348,9777,largeintestine,HPA,3000,3000,0.4,4,538473 13 541468 22 544463 30 547461 35 550459...,84.0,Male
349,9791,kidney,HPA,3000,3000,0.4,4,334733 33 337729 43 340729 43 343725 51 346723...,28.0,Male


In [3]:
train_data = TrainDataset(df_train=df_train, dataset_dir='../input/hubmap-organ-segmentation/')
print(train_data[0]['input'].shape)
print(train_data[0]['target'].shape)

torch.Size([3, 256, 256])
torch.Size([1, 256, 256])


In [4]:
pretrain_data = PreTrainDataset(df_train=df_train, dataset_dir='../input/hubmap-organ-segmentation/')
pretrain_data[0]['input'].shape

torch.Size([3, 128, 128])

In [5]:
pretrain_data_train, pretrain_data_test = split_dataset(pretrain_data, 0.97)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:
from torch.utils.data import DataLoader

pretrain_loader_train = DataLoader(pretrain_data_train, shuffle=True, batch_size=10)
pretrain_loader_test = DataLoader(pretrain_data_test, shuffle=True, batch_size=10)
train_loader = DataLoader(train_data,shuffle=True, batch_size=10)

In [8]:
base_model = TransUNet().to(device)
optimizer = torch.optim.Adam(base_model.parameters(), lr = 3e-4)
criterion = nn.BCELoss()

train(base_model, optimizer, pretrain_loader_train, criterion, device, 15)
train(base_model, optimizer, train_loader, criterion, device, 10)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [9]:
trsh = 0.3
trsh

0.3

In [10]:
import pandas as pd
df_test = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
df_test


,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness
0,10078,spleen,Hubmap,2023,2023,0.4945,4


In [11]:
df_test = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
base_model.eval()
test_dataset = TestDataset(df_test=df_test, dataset_dir="../input/hubmap-organ-segmentation/")
rle_test = get_rle_predictions(base_model, test_dataset, device, df_test['img_height'], df_test['img_width'], trsh)
rle_test

['81569 8 83592 8 85615 8 87638 8 89661 8 91684 8 93707 8 95722 24 97745 24 99768 24 101791 24 103814 24 105837 24 107860 24 109883 24 111906 24 113929 24 115952 24 117975 24 119998 24 122021 24 124044 24 126067 24 192794 16 194817 16 196840 16 198863 16 200886 16 202909 16 204932 16 206955 16 208971 23 210994 23 213017 23 215040 23 217063 23 219086 23 221109 23 223132 23 225155 23 227178 23 229201 23 231224 23 233247 23 235270 23 237293 23 239316 23 241331 8 241346 16 243354 8 243369 16 245377 8 245392 16 247400 8 247415 16 249423 8 249438 16 251446 8 251461 16 253469 8 253484 16 255492 23 257515 23 259538 23 261561 23 263584 23 265607 23 267630 23 269653 23 271676 8 273699 8 275722 8 277745 8 279768 8 281791 8 283814 8 285837 8 431714 16 433737 16 435760 16 437783 16 439806 16 441829 16 443852 16 445875 16 447906 16 449929 16 451952 16 453975 16 455998 16 458021 16 460044 16 462067 16 464090 16 466113 16 468136 16 470159 16 472182 16 474205 16 476228 16 478251 16 495715 8 497738 8 49

In [12]:
df_submission = pd.read_csv('../input/hubmap-organ-segmentation/sample_submission.csv')
df_submission['rle'] = rle_test
df_submission.to_csv('submission.csv', index=False)